In [3]:
import pandas as pd

# Load csv files into DataFrames
top_songs = pd.read_csv('../data/interim/top_songs_by_year.csv')
all_songs = pd.read_csv('../data/interim/all_songs.csv')
spotify_songs = pd.read_csv('../data/raw/spotify_dataset.csv')
print(top_songs.info())
print('*****************************************************************')
print(all_songs.info())
print('*****************************************************************')
print(spotify_songs.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playlist  1026 non-null   object
 1   track_id  1026 non-null   int64 
 2   year      1026 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 24.2+ KB
None
*****************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1943 entries, 0 to 1942
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Track ID             1943 non-null   int64  
 1   Title                1943 non-null   object 
 2   Artist               1467 non-null   object 
 3   Composer             1372 non-null   object 
 4   Album                1452 non-null   object 
 5   Genre                1433 non-null   object 
 6   Release Date         747 non-null    object 
 7   Kind                

In [5]:
spotify_songs.columns

Index(['Artist(s)', 'song', 'text', 'Length', 'emotion', 'Genre', 'Album',
       'Release Date', 'Key', 'Tempo', 'Loudness (db)', 'Time signature',
       'Explicit', 'Popularity', 'Energy', 'Danceability', 'Positiveness',
       'Speechiness', 'Liveness', 'Acousticness', 'Instrumentalness',
       'Good for Party', 'Good for Work/Study',
       'Good for Relaxation/Meditation', 'Good for Exercise',
       'Good for Running', 'Good for Yoga/Stretching', 'Good for Driving',
       'Good for Social Gatherings', 'Good for Morning Routine',
       'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
       'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
       'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'],
      dtype='object')

In [3]:
# Standardizing column names
all_songs = all_songs.rename(columns={"Track ID": "track_id"})
spotify_songs = spotify_songs.rename(columns={"apple_id": "track_id"})

# Merge the first two dataframes
merged_df = top_songs.merge(all_songs, on="track_id", how="left", suffixes=('', '_apple'))

# Merge the result with the third dataframe
final_df = merged_df.merge(spotify_songs, on="track_id", how="left", suffixes=('', '_spotify'))

# Display final structure
print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   playlist             1026 non-null   object 
 1   track_id             1026 non-null   int64  
 2   year                 1026 non-null   int64  
 3   Title                1026 non-null   object 
 4   Artist               656 non-null    object 
 5   Composer             597 non-null    object 
 6   Album                656 non-null    object 
 7   Genre                656 non-null    object 
 8   Release Date         332 non-null    object 
 9   Kind                 636 non-null    object 
 10  Total Time (ms)      656 non-null    float64
 11  Date Modified        656 non-null    object 
 12  Date Added           421 non-null    object 
 13  Release Date (Raw)   332 non-null    object 
 14  Date Modified (Raw)  656 non-null    object 
 15  Date Added (Raw)     421 non-null    o

In [4]:
# Prioritizing non-null values for Artist and Title
final_df["Artist"] = final_df["Artist"].fillna(final_df["artist"])
final_df["Title"] = final_df["Title"].fillna(final_df["title"])

# Selecting the required columns
filtered = final_df.loc[:, ["playlist", "track_id", "year", "Title", "Artist", "Album", "Genre", "spotify_id", "popularity"]]

# Display the result
print(filtered.info())
print(filtered.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   playlist    1026 non-null   object 
 1   track_id    1026 non-null   int64  
 2   year        1026 non-null   int64  
 3   Title       1026 non-null   object 
 4   Artist      1012 non-null   object 
 5   Album       656 non-null    object 
 6   Genre       656 non-null    object 
 7   spotify_id  975 non-null    object 
 8   popularity  975 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 72.3+ KB
None
      playlist  track_id  year  \
0  Replay 2015      1424  2015   
1  Replay 2015      1412  2015   
2  Replay 2015      3587  2015   
3  Replay 2015      1506  2015   
4  Replay 2015      2247  2015   

                                               Title           Artist  \
0  Highway Don't Care (feat. Taylor Swift & Keith...       Tim McGraw   
1                       

In [11]:
# Find the most common genre for each artist
artist_genre_map = filtered.groupby("Artist")["Genre"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)


In [17]:
# Fill missing genres using the most common genre per artist
filtered["Genre2"] = filtered["Genre"].fillna(filtered["Artist"].map(artist_genre_map))
filtered.to_csv('../data/interim/dataset.csv')